# Import libraries and data upload

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [105]:
test = pd.read_csv(r'C:\Users\USER\OneDrive\Documents\Property_prices_predictions\test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


# Data cleaning

In [106]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [107]:
test.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [108]:
# Drop columns that were dropped in the train dataset for uniformity
test.drop(columns=['Fence','Alley','MiscFeature','PoolQC'], inplace=True, errors='ignore')

In [109]:
# handling missing values similar to train
def fill_missing(df, column, value):
    if column in df.columns:
        df[column] = df[column].fillna(value)
        print(f"Filled {column} with {value}")

# Fireplace
if 'FireplaceQu' in test.columns:
    test['FireplaceQu'] = test['FireplaceQu'].fillna("Zero")

# Garage
garage_cat_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for col in garage_cat_cols:
    if col in test.columns:
        test[col] = test[col].fillna("NF")

if {'GarageYrBlt','GarageType','YearBuilt'}.issubset(test.columns):
    no_garage = test['GarageType'].isna() | (test['GarageType'] == 'NF')
    mask = ~no_garage & test['GarageYrBlt'].isna()
    test.loc[mask, 'GarageYrBlt'] = test.loc[mask, 'YearBuilt']
    test.loc[no_garage, 'GarageYrBlt'] = test.loc[no_garage, 'GarageYrBlt'].fillna(0)

fill_missing(test, 'GarageCars', 0)
fill_missing(test, 'GarageArea', 0)

# Basement
bsmt_cat = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']
for col in bsmt_cat:
    if col in test.columns:
        test[col] = test[col].fillna("NF")

if 'BsmtExposure' in test.columns and 'TotalBsmtSF' in test.columns:
    mode_exp = test.loc[test['BsmtExposure'] != "NF", 'BsmtExposure'].mode()
    if len(mode_exp)>0:
        mode_exp = mode_exp[0]
        test.loc[(test['BsmtExposure'].isna()) & (test['TotalBsmtSF'] > 0), 'BsmtExposure'] = mode_exp

if 'BsmtFinType2' in test.columns and 'TotalBsmtSF' in test.columns:
    mode_fin2 = test.loc[test['BsmtFinType2'] != "NF", 'BsmtFinType2'].mode()
    if len(mode_fin2)>0:
        mode_fin2 = mode_fin2[0]
        test.loc[(test['BsmtFinType2'].isna()) & (test['TotalBsmtSF'] > 0), 'BsmtFinType2'] = mode_fin2

# Masonry veneer
if 'MasVnrType' in test.columns:
    test['MasVnrType'] = test['MasVnrType'].fillna("None")
if 'MasVnrArea' in test.columns:
    test['MasVnrArea'] = test['MasVnrArea'].fillna(0)

# LotFrontage
if {'LotFrontage','Neighborhood'}.issubset(test.columns):
    nb_med = test.groupby('Neighborhood')['LotFrontage'].transform('median')
    test['LotFrontage'] = test['LotFrontage'].fillna(nb_med)
    test['LotFrontage'] = test['LotFrontage'].fillna(test['LotFrontage'].mean())

# categorical fills
for col in ['Exterior1st','Exterior2nd','SaleType','KitchenQual','Functional','MSZoning','Utilities']:
    if col in test.columns:
        test[col] = test[col].fillna(test[col].mode()[0])

num_fill_zero = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']
for col in num_fill_zero:
    if col in test.columns:
        test[col] = test[col].fillna(0)

Filled GarageCars with 0
Filled GarageArea with 0


In [110]:
(test.isna().sum().sort_values().tail(10) / len(test)) * 100

OpenPorchSF      0.0
EnclosedPorch    0.0
3SsnPorch        0.0
ScreenPorch      0.0
PoolArea         0.0
MiscVal          0.0
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
dtype: float64

## Duplicates

In [111]:
test.duplicated().sum()

np.int64(0)

In [112]:
# encoding
# ordinal mapping
qual_mapping = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NF":0, "Zero":0}

ordinal_cols = ['ExterQual','ExterCond','BsmtQual','BsmtCond',
                'HeatingQC','KitchenQual','GarageQual','GarageCond','FireplaceQu']

for col in ordinal_cols:
    if col in test.columns:
        test[col] = test[col].fillna('NF').map(qual_mapping)

# One-hot encode
cat_cols = test.select_dtypes(include=['object']).columns.tolist()
nominal_cols = [c for c in cat_cols if c not in ordinal_cols]
if len(nominal_cols) > 0:
    test = pd.get_dummies(test, columns=nominal_cols, drop_first=True)

# Feature engineering

In [ ]:
test['TotalSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']
test['Total_Bathrooms'] = (test.get('FullBath',0) + 0.5 * test.get('HalfBath',0) +
                           test.get('BsmtFullBath',0) + 0.5 * test.get('BsmtHalfBath',0))
test['RemodAge'] = test['YrSold'] - test['YearRemodAdd'] if ('YrSold' in test.columns and 'YearRemodAdd' in test.columns) else 0
test['HouseAge'] = test['YrSold'] - test['YearBuilt'] if ('YrSold' in test.columns and 'YearBuilt' in test.columns) else 0
test['OverallQualArea'] = test.get('OverallQual',0) * test['TotalSF']
test['OverallCond'] = test.get('OverallCond', 0)
test['QualityScore'] = test['OverallQual'] * test['OverallCond']
test['TotalPorch'] = test.get('OpenPorchSF',0) + test.get('EnclosedPorch',0) + test.get('ScreenPorch',0) + test.get('3SsnPorch',0)

In [ ]:
# Drop same low-importance columns as train
drop_cols = ['MiscVal','PoolArea','3SsnPorch','LowQualFinSF','KitchenAbvGr',
             'MoSold','OverallCond','EnclosedPorch','ScreenPorch',
             'Fence','Alley','MiscFeature','Utilities']
drop_prefixes = ['Fence_','Alley_','MiscFeature_','Utilities_']

to_drop = [c for c in test.columns if c in drop_cols or any(c.startswith(p) for p in drop_prefixes)]
test.drop(columns=to_drop, inplace=True, errors='ignore')

In [116]:
test.shape

(1459, 202)

In [117]:
test.to_csv("processed_test.csv", index=False)